In [107]:
import torch
import os
from gan.models.small_models import Generator
from gan.config import TrainingConfig,SmallModelConfig
from gan.utils import tensor_to_level_str,check_playable,check_level_similarity

config = SmallModelConfig()
config.set_env()
device = torch.device("cuda")

generator = Generator(
    out_dim=config.input_shape[0],
    shapes=config.model_shapes,
    z_shape=(config.latent_size,),
    filters=config.generator_filters,
    use_linear4z2features_g=config.use_linear4z2features_g,
    use_self_attention=config.use_self_attention_g,
    use_conditional=config.use_conditional,
    use_deconv_g=config.use_deconv_g
).to(device)

# model_save_path = "/root/mnt/GVGAI-GAN/gan/checkpoints/_boot_baseline-161"
# model_dict = torch.load(os.path.join(model_save_path, "models_3500.tar"))

# model_save_path = "/root/mnt/GVGAI-GAN/gan/checkpoints/_only5-121"
# model_dict = torch.load(os.path.join(model_save_path, "models_1240.tar"))

# model_save_path = "/root/mnt/GVGAI-GAN/gan/checkpoints/_ours-152"
# model_dict = torch.load(os.path.join(model_save_path, "models_3800.tar"))

# model_save_path = "/root/mnt/GVGAI-GAN/gan/checkpoints/_ours2-166"
# model_dict = torch.load(os.path.join(model_save_path, "models_800.tar"))

# model_save_path = "/root/mnt/GVGAI-GAN/gan/checkpoints/_wo-l1-154"
# model_dict = torch.load(os.path.join(model_save_path, "models_3800.tar"))

model_save_path = "/root/mnt/GVGAI-GAN/gan/checkpoints/_wo-simfilter-155"
model_dict = torch.load(os.path.join(model_save_path, "models_3600.tar"))

generator.load_state_dict(model_dict["generator"])




<All keys matched successfully>

In [87]:
def check_level_hamming(level1: str, level2: str):
    hit = 0
    for c1, c2 in zip(level1, level2):
        if c1 == "\n":
            continue
        if c1 != c2:
            hit += 1
    return hit


def check_level_object_duprecated(level1: str, level2: str):
    hit = 0
    for c1, c2 in zip(level1, level2):
        if c1 == "\n":
            continue
        if c1 in ['g','+','A'] and c1 == c2:
            hit += 1
    return hit==3


In [103]:
latents_for_eval = torch.randn(
    10000,
    config.latent_size,
).to(device)

output_levels = generator(latents_for_eval)
level_strs = tensor_to_level_str(config.env_name, output_levels)
playable_levels = []
playable_count = 0
for level_str in level_strs:
    if check_playable(level_str):
        playable_levels.append(level_str)

print("Playable Ratio:",len(playable_levels)/10000)


latents_for_eval = torch.randn(
    1000,
    config.latent_size,
).to(device)
output_levels = generator(latents_for_eval)
level_strs = tensor_to_level_str(config.env_name, output_levels)
playable_levels = []
playable_count = 0
for level_str in level_strs:
    if check_playable(level_str):
        playable_levels.append(level_str)
    
sum = 0
obj_sum = 0
n = 0
for i in range(len(playable_levels)):
    for j in range(len(playable_levels)):
        if i==j:
            continue
        obj_sum += check_level_object_duprecated(playable_levels[i],playable_levels[j])
        sum += check_level_hamming(playable_levels[i],playable_levels[j])
        n += 1
print("Hamming Dist:",sum/n)
print("Obj Duplication ratio",obj_sum/n)

Playable Ratio: 0.841


KeyboardInterrupt: 

In [108]:
playable_levels = []
while True:
    latents_for_eval = torch.randn(
        10000,
        config.latent_size,
    ).to(device)

    output_levels = generator(latents_for_eval)
    level_strs = tensor_to_level_str(config.env_name, output_levels)
    playable_count = 0
    for level_str in level_strs:
        if check_playable(level_str):
            playable_levels.append(level_str)
            if len(playable_levels) == 10000:
                break
    print(len(playable_levels))
    if len(playable_levels) == 10000:
        break
unique_levels = list(set(playable_levels))
print("Duplication Ratio:", 1-len(unique_levels)/len(playable_levels))


8063
10000
Duplication Ratio: 0.07989999999999997


In [89]:
n_w = 0
n_f = 0
n_e = 0
n = 0
for level in playable_levels:
    n += 1
    for c in level:
        if c == 'w':
            n_w += 1
        elif c == '.':
            n_f += 1
        elif c in ['1','2','3']:
            n_e += 1

print('Ave. Wall:', n_w/n)
print('Ave. Floor:', n_f/n)
print('Ave. Enemy:', n_e/n)


Ave. Wall: 77.40324594257179
Ave. Floor: 106.1960049937578
Ave. Enemy: 5.400749063670412


In [90]:
import glob

files = glob.glob("/root/mnt/GVGAI-GAN/gan/data/level/zelda/generated_good/*")

real_levels = []
for file in files:
    with open(os.path.join("/root/mnt/GVGAI-GAN/gan/data/level/zelda/generated_good/",file),'r') as f:
        s = f.read()
        real_levels.append(s)
hamming = 0
n = 0
for i in range(0, len(real_levels)):
    for j in range(i + 1, len(real_levels)):
        hamming += check_level_hamming(
            real_levels[i], real_levels[j])
        n += 1
print(hamming/(n)/(12*16))

ZeroDivisionError: division by zero

In [ ]:
import glob

files = glob.glob("/root/mnt/GVGAI-GAN/gan/data/level/zelda/generated_large/*")

real_levels = []
for file in files:
    with open(os.path.join("/root/mnt/GVGAI-GAN/gan/data/level/zelda/generated_large/", file), 'r') as f:
        s = f.read()
        real_levels.append(s)
hamming = 0
n = 0
for i in range(0, len(real_levels)):
    for j in range(i + 1, len(real_levels)):
        hamming += check_level_hamming(
            real_levels[i], real_levels[j])
        n += 1
print(hamming/(n)/(12*16))


0.12806500171009358


In [ ]:
import glob

files = glob.glob("/root/mnt/GVGAI-GAN/gan/data/level/zelda/originals/*")

real_levels = []
for file in files:
    with open(os.path.join("/root/mnt/GVGAI-GAN/gan/data/level/zelda/originals/", file), 'r') as f:
        s = f.read()
        real_levels.append(s)
hamming = 0
n = 0
for i in range(0, len(real_levels)):
    for j in range(i + 1, len(real_levels)):
        hamming += check_level_hamming(
            real_levels[i], real_levels[j])
        n += 1
print(hamming/(n)/(12*16))

n_w = 0
n_f = 0
n_e = 0
n = 0
for level in real_levels:
    n += 1
    for c in level:
        if c == 'w':
            n_w += 1
        elif c == '.':
            n_f += 1
        elif c in ['1', '2', '3']:
            n_e += 1

print('Ave. Wall:', n_w/n)
print('Ave. Floor:', n_f/n)
print('Ave. Enemy:', n_e/n)


0.2265625
Ave. Wall: 71.4
Ave. Floor: 114.4
Ave. Enemy: 3.2


In [ ]:
import glob

files = glob.glob("/root/mnt/GVGAI-GAN/gan/data/level/zelda/random/*")

real_levels = []
for file in files:
    with open(os.path.join("/root/mnt/GVGAI-GAN/gan/data/level/zelda/random/", file), 'r') as f:
        s = f.read()
        real_levels.append(s)
hamming = 0
n = 0
for i in range(0, len(real_levels)):
    for j in range(i + 1, len(real_levels)):
        hamming += check_level_hamming(
            real_levels[i], real_levels[j])
        n += 1
print(hamming/(n)/(12*16))


0.24803135521885522


In [ ]:
if config.cuda:
    device = torch.device(
        "cuda" if torch.cuda.is_available else "cpu")
    print("device : cuda")
else:
    device = torch.device("cpu")
    print("device : cpu")
generator = Generator(
    out_dim=config.input_shape[0],
    shapes=config.model_shapes,
    z_shape=(config.latent_size,),
    filters=config.generator_filters,
    use_conditional=config.use_conditional
).to(device)

model_path = os.path.join(
    "/root/mnt/pcg/GVGAI-GAN/gan/checkpoints/none-795", "models.tar")
load_model = torch.load(model_path)
generator.load_state_dict(load_model["generator"])

x = torch.randn(config.latent_size).to(device)

# es = cma.CMAEvolutionStrategy(config.latent_size*[0], 0.5)
# es.optimize(fitness)

def fitness(x: torch.Tensor):
    x = np.array(x)
    latent = torch.FloatTensor(x).view(1, -1, 1, 1)
    level = generator(x)
    level_str = tensor_to_level_str(level)
    playable = check_playable(level_str)
    eval = eval(level_str)
    return playable*300 + eval

def eval(level_str):
    wall = 0
    enemy = 0
    for s, i in enumerate(level_str):
        for c, j in enumerate(level_str):
            if c == 'w':
                wall += 1
            if c in ['1', '2', '3']:
                enemy += 1
    return wall + enemy


device : cuda


TypeError: __init__() got an unexpected keyword argument 'use_conditional'